# Building a Database for Crime Reports

In this project we will be building a database that stores Boston crimes in a Postgres database. 

## Creating the Crime Database

In [2]:
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.commit()
conn.close()

In [3]:
conn = psycopg2.connect("dbname=crime_db user=dq")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")
conn.commit()

## Obtaining the Column Names and Sample

In [4]:
import csv
with open("boston.csv") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

## Creating an Auxiliary Function


In [5]:
def get_col_value_set(csv_filename, col_index):
    import csv
    values = set()
    with open(csv_filename, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_value_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


# Finding the Maximum Length

In [6]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [7]:
descriptions = get_col_value_set("boston.csv", 2)
max_length = 0
for description in descriptions:
    max_length = max(max_length,len(description))
print(max_length)

58


# Creating the Table

In [8]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [9]:
print(first_row)

['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


column 1: Incident Number

Type: integer  

column 2: Offense Code

Type: smallint

column 3: Description

Type: varchar(58)

column 4: Date

Type: DATE

column 5: Day of the Week

Type: day_of_week

column 6: Latitude

Type: DECIMAL(11, 8)

column 7: Longitude

Type: DECIMAL(11, 8)

In [10]:
cur.execute("CREATE TYPE day_of_week AS ENUM ('Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday');")

In [11]:
cur.execute("CREATE TABLE crimes.boston_crimes (Incident_Number INTEGER PRIMARY KEY, Offense_Code smallint, Description varchar(58), Date DATE, Day_of_the_Week day_of_week, Latitude DECIMAL(11,8), Longitude DECIMAL(11,8));")

## Loading the Data

Let us now load the data from the 'boston.csv' into our newly generated table on our Postgres database.

In [12]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")
# print the number of rows to ensure that they were loaded
print(len(cur.fetchall()))

298329


## Revoking Public Privileges


In [13]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating User Groups



In [14]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")


cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")


cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

## Testing

In [15]:
# close the old connection to test with a brand new connection
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()
# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
